In [23]:
import pandas as pd
import time
import numpy as np

In [27]:
csvfile = '../clean_data/cleaned_get_it_done_open_potholes.csv'
df = pd.read_csv(csvfile)

df = df.drop(['address', 'sap_notification_number', 'sap_problem_code', 'service_subtype', 'source', \
            'referred_email_update', 'referral_email', 'functional_location', 'description', \
            'agency_responsible' , 'sap_problem_category', 'case_record_type', 'updated_datetime',\
            'service_request_id', 'parent_case_number'], axis = 1)

#drop all potholes 
df = df.drop(df[df.sap_problem_type != 'Pothole'].index)

#drop all NaNs for Long and Lat
df = df.dropna(subset = ['long'])
df = df.dropna(subset = ['lat'])

#drop all non-active potholes
df = df.drop(df[df.status_description == 'Duplicate'].index)
df = df.drop(df[df.status_description== 'Closed'].index)
df = df.drop(df[df.status_description=='Closed - Referred'].index)

#Drop all values that are open
df = df.dropna(subset = ['closed_date'])


In [22]:
neighborhoods = np.zeros((len(df['lat']),1))
df['Neighborhood'] = neighborhoods

# Vinh : 0-2400
# Lee Anne: 2400 - 4800
# Maggie: 

In [20]:
#get neighborhoods of potholes
import geocoder 

neighborhood = []

for i in range (11,19):
    latitutde = df.iloc[[i]]['lat']
    longitude = df.iloc[[i]]['long']
    time.sleep(.1)
    g = geocoder.google([latitutde, longitude], method = 'reverse').json
    if 'neighborhood' not in g:
         df.ix[i, 'Neighborhood'] = 'NaN'
    else:
        neighb = g['neighborhood']
        df.ix[i, 'Neighborhood'] = neighb
    
# for i in range (1401, len(df['long'])):
#     neighborhood.append('ADD ME')
# df['Neighborhood'] = neighborhood

# NAME THIS AS '../clean_data/0-2500' 
# or whatever you're range was

In [21]:
df.to_csv('../clean_data/')

,Unnamed: 0,status_description,mobile_web_status,duplicate_verified,override_duplicate,requested_datetime,closed_date,open,closed,service_name,coordinates,lat,long,sap_problem_type,Neighborhood
0,1,Acknowledged,In process,0,0,2017-05-23T18:06:00,NaN,1,0,Pothole,32.79153673521502-117.1544678834988,32.791537,-117.154468,Pothole,Birdland
1,54,New,New,0,0,2017-05-23T15:31:00,NaN,1,0,Pothole,32.724165-117.05552499999999,32.724165,-117.055525,Pothole,Encanto
2,60,Acknowledged,In process,0,0,2017-05-23T15:25:00,NaN,1,0,Pothole,32.79042917715904-117.24192641030152,32.790429,-117.241926,Pothole,Pacific Beach
3,78,Acknowledged,In process,0,0,2017-05-23T14:43:00,NaN,1,0,Pothole,32.760451-117.064381,32.760451,-117.064381,Pothole,Mid-City
4,81,New,New,0,0,2017-05-23T14:40:00,NaN,1,0,Pothole,32.758759-117.063192,32.758759,-117.063192,Pothole,Mid-City
5,88,Acknowledged,In process,0,0,2017-05-23T14:17:00,NaN,1,0,Pothole,32.748611-117.079278,32.748611,-117.079278,Pothole,Mid-City
6,116,Acknowledged,In process,0,1,2017-05-23T12:52:00,NaN,1,0,Pothole,32.844727311545895-117.2670286848815,32.844727,-117.267029,Pothole,La Jolla
7,121,Acknowledged,In process,0,0,2017-05-23T12:45:00,NaN,1,0,Pothole,32.74642288029995-117.1482656743188,32.746423,-117.148266,Pothole,Hillcrest
8,125,Acknowledged,In process,0,1,2017-05-23T12:42:00,NaN,1,0,Pothole,32.74642295486662-117.1493714079478,32.746423,-117.149371,Pothole,Hillcrest
9,127,Acknowledged,In process,0,0,2017-05-23T12:38:00,NaN,1,0,Pothole,32.74513698504784-117.1605940669536,32.745137,-117.160594,Pothole,Hillcrest


In [315]:
from datetime import datetime
date_format = "%Y-%m-%d %H:%M:%S"

In [316]:
for i in df.index:
    closed_date = str(df['closed_date'][i])
    open_date = str(df['requested_datetime'][i])
    closed_date = closed_date.replace('T',' ')
    open_date = open_date.replace('T',' ')
    df.ix[i, 'closed_date'] = closed_date
    df.ix[i, 'requested_datetime'] = open_date


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


'2017-05-23 08:41:00'

In [327]:
time_list = []

for i in df.index:
    a = datetime.strptime(str(df['closed_date'][i]), date_format)
    b = datetime.strptime(str(df['requested_datetime'][i]), date_format)
    delta = a - b
    hours_seconds = delta.seconds/3600
    hours_days = delta.days * 24
    time_list.append(hours_days + hours_seconds)
df['Duration to Close (Hrs)'] = time_list

In [326]:
a = datetime.strptime(str(df['closed_date'][6125]), date_format)
b = datetime.strptime(str(df['requested_datetime'][6125]), date_format)
delta = a - b
days = delta.minutes
print(a)
print(b)
print(days)

AttributeError: 'datetime.timedelta' object has no attribute 'minutes'

[0, 0, 0, 0, 0, 0, 0, 0, 0]